In [3]:
### data
import pandas as pd
import numpy as np

### APIs
import tweepy

## Plots
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sea
get_ipython().magic('matplotlib inline')

# Sentiment analysis
from textblob import TextBlob
import re

In [45]:
with open(os.path.join(os.getcwd(),"data/credentials.json")) as data_file:    
    key = json.load(data_file)

#print key["API_KEY"], key["API_SECRET"], key["ACCESS_TOKEN"], key["ACCESS_TOKEN_SECRET"]

In [46]:
def twitter_setup():
    #authentication
    auth = tweepy.OAuthHandler(key["API_KEY"], key["API_SECRET"])
    auth.set_access_token(key["ACCESS_TOKEN"], key["ACCESS_TOKEN_SECRET"])    
    return tweepy.API(auth)

extractor = twitter_setup()



In [47]:
tweets = extractor.user_timeline(screen_name="cdvel", count=100)
print ("No. tweets extracted: {}\n".format(len(tweets)))

No. tweets extracted: 100



In [48]:
print ("Latest 5 tweets")
for tweet in tweets[:5]:
    print (tweet.text)
    print()


Latest 5 tweets
LG has distributed its AI-based platform, DeepThinQ 1.0 to other products as well https://t.co/ewcI9PUo9D via @reddit
()
How to tame the tech titans - Competition in the digital age https://t.co/toGDcyS3oh
()
Microsoft execs say in 20 years we'll all have digital assistants that will be our alter egos — and we need to set https://t.co/UvYjDZ3n9q
()
Facebook's next news feed tweak: ranking 'trusted' sources https://t.co/DG1Nm5QJ3V
()
Is video really the future of VR? https://t.co/jvafrWeKtV
()


In [70]:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
data['length']  = np.array([len(tweet.text) for tweet in tweets])
data['id']   = np.array([tweet.id for tweet in tweets])
data['created'] = np.array([tweet.created_at for tweet in tweets])
data['source'] = np.array([tweet.source for tweet in tweets])
data['no_likes'] = np.array([tweet.favorite_count for tweet in tweets])
data['no_retweets'] = np.array([tweet.retweet_count for tweet in tweets])
data['slug'] = np.array([''.join([tweet.user.screen_name, '/status/', str(tweet.id)])  for tweet in tweets])


#print tweet
display(data.head(10))

,Tweets,length,id,created,source,no_likes,no_retweets,link
0,"LG has distributed its AI-based platform, Deep...",117,955272971609817088,2018-01-22 02:56:39,dlvr.it,0,0,cdvel/status/955272971609817088
1,How to tame the tech titans - Competition in t...,84,955122950776283136,2018-01-21 17:00:31,Buffer,0,0,cdvel/status/955122950776283136
2,Microsoft execs say in 20 years we'll all have...,138,955091407093542912,2018-01-21 14:55:10,Crowdfire - Go Big,1,0,cdvel/status/955091407093542912
3,Facebook's next news feed tweak: ranking 'trus...,82,955088894281830400,2018-01-21 14:45:11,Crowdfire - Go Big,1,0,cdvel/status/955088894281830400
4,Is video really the future of VR? https://t.co...,57,955085500251017217,2018-01-21 14:31:42,Crowdfire - Go Big,0,0,cdvel/status/955085500251017217
5,Top 5 VR180 videos you can watch on YouTube wi...,81,954871964115652609,2018-01-21 00:23:11,Crowdfire - Go Big,1,0,cdvel/status/954871964115652609
6,RT @WIRED: These tech books are worth your mon...,72,954711900201390085,2018-01-20 13:47:09,Twitter for Android,0,50,cdvel/status/954711900201390085
7,"RT @WIRED: ""It was the second most exciting pa...",133,954567688587628544,2018-01-20 04:14:06,Twitter for Android,0,1327,cdvel/status/954567688587628544
8,How Libratus beat top poker pros for 1.5 milli...,140,954548170675036160,2018-01-20 02:56:33,dlvr.it,0,0,cdvel/status/954548170675036160
9,Listening to: Voices of AI with Kent Bye: #1: ...,131,954200566627942400,2018-01-19 03:55:17,Twitter for Android,0,0,cdvel/status/954200566627942400


In [72]:
mean_length = np.mean(data['length'])

print("Tweet's avg length: {}".format(mean_length))

likes_max = np.max(data['no_likes'])
retweets_max = np.max(data['no_retweets'])

most_likes = data[data.no_likes == likes_max].index[0]
most_retweets = data[data.no_retweets == retweets_max].index[0]


print ("\nMost liked: \n> {} length={} ♡={}".format(data['Tweets'][most_likes].encode('utf-8'), data['length'][most_likes], likes_max))


print ("\nMost retweeted: \n> {} length={} RTs={}".format(data['Tweets'][most_retweets].encode('utf-8'), data['length'][most_retweets], retweets_max))



Tweet's avg length: 94.96

Tweet with most likes: 
> Microsoft execs say in 20 years we'll all have digital assistants that will be our alter egos — and we need to set https://t.co/UvYjDZ3n9q length=138 ♡=1

Tweet most retweeted: 
> RT @WIRED: "It was the second most exciting paper I had read in the blockchain era. The first was Satoshi’s." https://t.co/h56lb6RJok length=133 RTs=1327


### References 
- https://github.com/avisaxena33/Twitter-Data-Sentiment-Analysis-with-Python-Workshop/blob/master/twitter.py

### Notes
oauthlib, textblob for OSX64
- https://anaconda.org/asmeurer/oauthlib
- https://anaconda.org/sloria/textblob
    